## Assignment - Data Exploration 6

To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:

1. Are there any differences between the genders, ethnicities, and parental level of education with respect to their performances in exams?
2. Are there any differences between the lunch types with respect to their performances in exams? If there are, how do you explain this?
3. Does the test preparation course seem to have an effect on the exam performances?
4. Which 2 exam scores are most correlated with each other?

In [47]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats.stats import ttest_ind
from sqlalchemy import create_engine
import warnings

warnings.filterwarnings('ignore')
sns.set(style="darkgrid")

In [48]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'studentsperformance'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

STUDENTS = pd.read_sql_query('select * from studentsperformance',con=engine)

# no need for an open connection, 
# as we're only doing a single query
engine.dispose()

In [123]:
STUDENTS.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [124]:
TARGETS = ['math score', 'reading score', 'writing score']

## Assignment - Data Exploration 6.1

Are there any differences between the genders, ethnicities, and parental level of education with respect to their performances in exams?

In [122]:
STUDENTS.groupby("gender").mean()[TARGETS].transpose()

gender,female,male
math score,63.633205,68.728216
reading score,72.608108,65.473029
writing score,72.467181,63.311203


The mean scores are different, but are they significant?

In [100]:
# function takes a df, a categorical feature vector, a single numeric feature vectors of list of numeric feature vectors
# and prints t statistics and p-values of variables

def cat_num_compare(df, categorical, numeric_col):
    ETHY = df[categorical].unique()
    for var in numeric_col:
        print('-'*60)
        print('Comparisions for variable: {}'.format(var))
        print('-'*60)
        for i in range(0, len(ETHY)):
            for j in range(i+1, len(ETHY)):
                print('t-test between groups {0} and {1}:'.format(ETHY[i], ETHY[j]))
                print(ttest_ind(df[df[categorical] == ETHY[i]][var],
                               df[df[categorical] == ETHY[j]][var]))
    return

In [103]:
cat_num_compare(STUDENTS, 'gender', TARGETS)

------------------------------------------------------------
Comparisions for variable: math score
------------------------------------------------------------
t-test between groups female and male:
Ttest_indResult(statistic=-5.383245869828983, pvalue=9.120185549328822e-08)
------------------------------------------------------------
Comparisions for variable: reading score
------------------------------------------------------------
t-test between groups female and male:
Ttest_indResult(statistic=7.959308005187657, pvalue=4.680538743933289e-15)
------------------------------------------------------------
Comparisions for variable: writing score
------------------------------------------------------------
t-test between groups female and male:
Ttest_indResult(statistic=9.979557910004507, pvalue=2.019877706867934e-22)


Based on the low p-values returned by the t-test, it appears that the difference in mean test scores between these two groups(gender) are statistically significant.

In [121]:
STUDENTS.groupby("race/ethnicity").describe()[TARGETS].transpose()

race/ethnicity          group A     group B     group C     group D  \
math score    count   89.000000  190.000000  319.000000  262.000000   
              mean    61.629213   63.452632   64.463950   67.362595   
              std     14.523008   15.468191   14.852666   13.769386   
              min     28.000000    8.000000    0.000000   26.000000   
              25%     51.000000   54.000000   55.000000   59.000000   
              50%     61.000000   63.000000   65.000000   69.000000   
              75%     71.000000   74.000000   74.000000   77.000000   
              max    100.000000   97.000000   98.000000  100.000000   
reading score count   89.000000  190.000000  319.000000  262.000000   
              mean    64.674157   67.352632   69.103448   70.030534   
              std     15.543762   15.177499   13.997033   13.895306   
              min     23.000000   24.000000   17.000000   31.000000   
              25%     53.000000   56.000000   60.000000   60.250000   
              50%     64.000000   67.000000   71.000000   71.000000   
              75%     74.000000   79.750000   78.500000   79.000000   
              max    100.000000   97.000000  100.000000  100.000000   
writing score count   89.000000  190.000000  319.000000  262.000000   
              mean    62.674157   65.600000   67.827586   70.145038   
              std     15.468278   15.625173   14.983378   14.367707   
              min     19.000000   15.000000   10.000000   32.000000   
              25%     51.000000   55.250000   57.000000   61.000000   
              50%     62.000000   67.000000   68.000000   72.000000   
              75%     73.000000   78.000000   79.000000   80.000000   
              max     97.000000   96.000000  100.000000  100.000000   

race/ethnicity          group E  
math score    count  140.000000  
              mean    73.821429  
              std     15.534259  
              min     30.000000  
              25%     64.750000  
              50%     74.500000  
              75%     85.000000  
              max    100.000000  
reading score count  140.000000  
              mean    73.028571  
              std     14.874024  
              min     26.000000  
              25%     63.000000  
              50%     74.000000  
              75%     84.000000  
              max    100.000000  
writing score count  140.000000  
              mean    71.407143  
              std     15.113906  
              min     22.000000  
              25%     62.000000  
              50%     72.000000  
              75%     80.250000  
              max    100.000000

I am noticing some difference in mean test scores grouped by race, but lets test to see if they are significant.

In [101]:
cat_num_compare(STUDENTS, 'race/ethnicity', TARGETS)

------------------------------------------------------------
Comparisions for variable: math score
------------------------------------------------------------
t-test between groups group B and group C:
Ttest_indResult(statistic=-0.7315669893534263, pvalue=0.4647708939167453)
t-test between groups group B and group A:
Ttest_indResult(statistic=0.9355076279747488, pvalue=0.3503415961659957)
t-test between groups group B and group D:
Ttest_indResult(statistic=-2.82845539712675, pvalue=0.004886197137104194)
t-test between groups group B and group E:
Ttest_indResult(statistic=-6.007397050552227, pvalue=5.007946047497971e-09)
t-test between groups group C and group A:
Ttest_indResult(statistic=1.5997220303217299, pvalue=0.11043810745588042)
t-test between groups group C and group D:
Ttest_indResult(statistic=-2.41858624746011, pvalue=0.015888349556016285)
t-test between groups group C and group E:
Ttest_indResult(statistic=-6.127642520822135, pvalue=1.927238849552764e-09)
t-test between gro

# ADD note to determine significance

In [120]:
STUDENTS.groupby("parental level of education").describe()[TARGETS].transpose()

parental level of education  associate's degree  bachelor's degree  \
math score    count                  222.000000         118.000000   
              mean                    67.882883          69.389831   
              std                     15.112093          14.943789   
              min                     26.000000          29.000000   
              25%                     57.000000          61.000000   
              50%                     67.000000          68.000000   
              75%                     80.000000          79.000000   
              max                    100.000000         100.000000   
reading score count                  222.000000         118.000000   
              mean                    70.927928          73.000000   
              std                     13.868948          14.285250   
              min                     31.000000          41.000000   
              25%                     61.000000          63.000000   
              50%                     72.500000          73.000000   
              75%                     81.000000          82.750000   
              max                    100.000000         100.000000   
writing score count                  222.000000         118.000000   
              mean                    69.896396          73.381356   
              std                     14.311122          14.728262   
              min                     35.000000          38.000000   
              25%                     58.000000          62.500000   
              50%                     70.500000          74.000000   
              75%                     80.000000          83.000000   
              max                    100.000000         100.000000   

parental level of education  high school  master's degree  some college  \
math score    count           196.000000        59.000000    226.000000   
              mean             62.137755        69.745763     67.128319   
              std              14.539651        15.153915     14.312897   
              min               8.000000        40.000000     19.000000   
              25%              53.750000        55.500000     59.000000   
              50%              63.000000        73.000000     67.500000   
              75%              72.000000        81.000000     76.000000   
              max              99.000000        95.000000    100.000000   
reading score count           196.000000        59.000000    226.000000   
              mean             64.704082        75.372881     69.460177   
              std              14.132130        13.775163     14.057049   
              min              24.000000        42.000000     23.000000   
              25%              54.000000        65.500000     60.000000   
              50%              66.000000        76.000000     70.500000   
              75%              74.250000        84.500000     79.750000   
              max              99.000000       100.000000    100.000000   
writing score count           196.000000        59.000000    226.000000   
              mean             62.448980        75.677966     68.840708   
              std              14.085907        13.730711     15.012331   
              min              15.000000        46.000000     19.000000   
              25%              52.000000        67.000000     60.000000   
              50%              64.000000        75.000000     70.000000   
              75%              73.000000        85.000000     79.000000   
              max             100.000000       100.000000     99.000000   

parental level of education  some high school  
math score    count                179.000000  
              mean                  63.497207  
              std                   15.927989  
              min                    0.000000  
              25%                   53.000000  
              50%                   65.000000  
              75%                   7

In [108]:
cat_num_compare(STUDENTS, 'parental level of education', TARGETS)

------------------------------------------------------------
Comparisions for variable: math score
------------------------------------------------------------
t-test between groups bachelor's degree and some college:
Ttest_indResult(statistic=1.3702352829617477, pvalue=0.17151224218471559)
t-test between groups bachelor's degree and master's degree:
Ttest_indResult(statistic=-0.14868196568849312, pvalue=0.8819757824867702)
t-test between groups bachelor's degree and associate's degree:
Ttest_indResult(statistic=0.8786641889793243, pvalue=0.38020757818036177)
t-test between groups bachelor's degree and high school:
Ttest_indResult(statistic=4.236143622484822, pvalue=2.9965345853843376e-05)
t-test between groups bachelor's degree and some high school:
Ttest_indResult(statistic=3.196718602676573, pvalue=0.0015409192922812146)
t-test between groups some college and master's degree:
Ttest_indResult(statistic=-1.2356334215824571, pvalue=0.21761956810347416)
t-test between groups some colleg

# ADD note about signifigance

## Assignment - Data Exploration 6.2

Are there any differences between the lunch types with respect to their performances in exams? If there are, how do you explain this?

In [119]:
STUDENTS.groupby("lunch").mean()[TARGETS].transpose()

lunch,free/reduced,standard
math score,58.921127,70.034109
reading score,64.653521,71.654264
writing score,63.022535,70.823256


In [116]:
cat_num_compare(STUDENTS, 'lunch', TARGETS)

------------------------------------------------------------
Comparisions for variable: math score
------------------------------------------------------------
t-test between groups standard and free/reduced:
Ttest_indResult(statistic=11.837180472914612, pvalue=2.4131955993137074e-30)
------------------------------------------------------------
Comparisions for variable: reading score
------------------------------------------------------------
t-test between groups standard and free/reduced:
Ttest_indResult(statistic=7.451056467473455, pvalue=2.0027966545279011e-13)
------------------------------------------------------------
Comparisions for variable: writing score
------------------------------------------------------------
t-test between groups standard and free/reduced:
Ttest_indResult(statistic=8.009784197834758, pvalue=3.186189583166477e-15)


# add note about results

## Assignment - Data Exploration 6.3

Does the test preparation course seem to have an effect on the exam performances?

In [117]:
STUDENTS.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [128]:
STUDENTS.groupby('test preparation course')[TARGETS].mean().transpose()

test preparation course,completed,none
math score,69.695531,64.077882
reading score,73.893855,66.534268
writing score,74.418994,64.504673


In [129]:
cat_num_compare(STUDENTS, 'test preparation course', TARGETS)

------------------------------------------------------------
Comparisions for variable: math score
------------------------------------------------------------
t-test between groups none and completed:
Ttest_indResult(statistic=-5.704616417349102, pvalue=1.5359134607147415e-08)
------------------------------------------------------------
Comparisions for variable: reading score
------------------------------------------------------------
t-test between groups none and completed:
Ttest_indResult(statistic=-7.871663538941468, pvalue=9.081783336892205e-15)
------------------------------------------------------------
Comparisions for variable: writing score
------------------------------------------------------------
t-test between groups none and completed:
Ttest_indResult(statistic=-10.409173436808748, pvalue=3.68529173524572e-24)


# add thoughts about results

## Assignment - Data Exploration 6.4

Which 2 exam scores are most correlated with each other?

In [110]:
STUDENTS[["math score", "reading score", "writing score"]].corr()

,math score,reading score,writing score
math score,1.000000,0.817580,0.802642
reading score,0.817580,1.000000,0.954598
writing score,0.802642,0.954598,1.000000


# add thoughts about results